In [2]:
    # 이미지 링크 가져오기
#     img_src = item.find_element_by_class_name('product_img').get_attribute('src')
    
    # 배경 색상 가져오기
#     bg_color = item.find_element_by_class_name('product').get_attribute('style')


In [1]:
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from webdriver_manager.chrome import ChromeDriverManager
import time
from tqdm import tqdm_notebook

# 크롤링

In [21]:
# 모든 페이지 불러오기 위해 스크롤 내리기

# 크롬 드라이버매니저 설치
driver = webdriver.Chrome(ChromeDriverManager().install())

# 크림 url 정보 입력
url = "https://kream.co.kr/search?sort=popular&per_page=20"

# 크림 url 가져오기
driver.get(url)

for i in range(10):
    time.sleep(1)
    driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")



====== WebDriver manager ======
Current google-chrome version is 98.0.4758
Get LATEST chromedriver version for 98.0.4758 google-chrome
Driver [C:\Users\teysi\.wdm\drivers\chromedriver\win32\98.0.4758.102\chromedriver.exe] found in cache


In [22]:

product_infos = []

# 페이지 내의 상품 리스트를 가져옴
items = driver.find_element_by_class_name('search_result_list')
item_list = items.find_elements_by_class_name('search_result_item')

for item in item_list:
    # 제목, 가격 등 기본 정보 가져오기
    info = item.find_element_by_class_name('item_inner')
    text = info.text.split("\n")

    #인덱싱으로 브랜드, 이름, 리셀가격, 빠른배송 여부 긁어오기
    if len(text) == 6:
        brand = text[0]
        en_name = text[1]
        kor_name = text[2]
        re_price = text[3][:-1]
        speed_deli = 1
    else:
        brand = text[0]
        en_name = text[1]
        kor_name = text[2]
        re_price = text[3][:-1]
        speed_deli = 0

    # 상품 세부 페이지 link 가져오기
    link = info.get_attribute('href')

    # 저장
    item_dict = {
        "brand": brand,
        "en_name": en_name,
        "kor_name": kor_name,
        "re_price": re_price,
        "speed_deli": speed_deli,
        'link': link
    }

    product_infos.append(item_dict)

df = pd.DataFrame(product_infos)


In [ ]:
product_detail = []
pro_num = 0
num = 0

for i in tqdm_notebook(range(len(df))):
    num+=1
    while pro_num < num:
        if pro_num == 361:
            break
        try:
            number = df["link"][i].split("/")[4]
            url = f"https://kream.co.kr/products/{number}"
            time.sleep(random.randint(1, 3))
            driver.get(url)

            product_items = driver.find_element_by_class_name('detail_product')
            items = product_items.text.split("\n")

            product_dict = {
                "link" : url,
                "model" : items[1],
                "release" : items[3],
                "color" : items[5],
                "price" : items[7]
            }
            product_detail.append(product_dict)
            pro_num += 1
            print(f"{i+1}번 크롤링 완료")
        except:
            driver.refresh()

df1 = pd.DataFrame(product_detail)

<ipython-input-24-5a53ca7c28d8>:5: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  for i in tqdm_notebook(range(len(df))):


  0%|          | 0/360 [00:00<?, ?it/s]

1번 크롤링 완료
2번 크롤링 완료
3번 크롤링 완료
4번 크롤링 완료
5번 크롤링 완료
6번 크롤링 완료
7번 크롤링 완료
8번 크롤링 완료
9번 크롤링 완료
10번 크롤링 완료
11번 크롤링 완료
12번 크롤링 완료
13번 크롤링 완료
14번 크롤링 완료
15번 크롤링 완료
16번 크롤링 완료
17번 크롤링 완료


# merge 및 전처리

In [8]:
def preprocessing(date):
    total = pd.merge(df, df1, on="link")
    total["product_num"] = total["link"].apply(lambda x: x.split("/")[4])
    total = total[["brand", "en_name", "kor_name", "product_num", "model", "release", "color", "price", "re_price", "link"]]
    total.to_csv(f"{date}_kream.csv", index = False)
    return total

# 작동 실행

In [56]:
preprocessing(220217)

,brand,en_name,kor_name,product_num,model,release,color,price,re_price,link
0,Jordan,Jordan 1 Retro High OG Black Mocha,조던 1 레트로 하이 OG 블랙 모카,25922,555088-105,20/11/13,SAIL/DARK MOCHA-BLACK-BLACK,"199,000원","690,000",https://kream.co.kr/products/25922
1,Jordan,Jordan 1 Mid Light Smoke Grey Anthracite,조던 1 미드 라이트 스모크 그레이 앤트러사이트,47257,554724-078,22/01/24,LIGHT SMOKE GREY/WHITE-ANTHRACITE,"€120 (약 163,200원)","338,000",https://kream.co.kr/products/47257
2,Nike,Nike Dunk Low Retro Championship Court Purple,나이키 덩크 로우 레트로 챔피언쉽 코트 퍼플,47887,DD1391-104,22/02/11,WHITE/COURT PURPLE,"129,000원","225,000",https://kream.co.kr/products/47887
3,Jordan,Jordan 1 High 85 College Navy,조던 1 하이 85 컬리지 네이비,51017,BQ4422-400,22/02/16,COLLEGE NAVY/SUMMIT WHITE-TECH GREY,"229,000원","620,000",https://kream.co.kr/products/51017
4,Nike,(W) Nike Dunk Low Black,(W) 나이키 덩크 로우 블랙,28030,DD1503-101,21/01/04,WHITE/BLACK,"129,000원","279,000",https://kream.co.kr/products/28030
...,...,...,...,...,...,...,...,...,...,...
355,Nike,Nike x Off-White Air Rubber Dunk University Gold,나이키 x 오프화이트 에어 러버 덩크 유니버시티 골드,25831,CU6015-700,20/10/01,UNIVERSITY GOLD/UNIVERSITY GOLD-BLACK,"229,000원","249,000",https://kream.co.kr/products/25831
356,Nike,(W) Nike Air Force 1 Fontanka Pearl White,(W) 나이키 에어포스 1 폰탄카 펄 화이트,43413,DA7024-200,21/10/18,PEARL WHITE/PEARL WHITE/UNIVERSITY RED/PEARL W...,"149,000원","249,000",https://kream.co.kr/products/43413
357,Pokemon,Pokemon Card Game Sword & Shield Expansion Pac...,포켓몬 카드 게임 소드&실드 확장팩 25주년 기념 컬렉션 박스 (16팩),51611,-,21/10/22,-,"40,000원","58,000",https://kream.co.kr/products/51611
358,Nike,Nike Air Force 1 '07 LV8 Paisley Swoosh,나이키 에어포스 1 '07 LV8 페이즐리 스우시,33544,DJ4631-200,21/04/23,RATTAN/PALE IVORY/BLACK,"139,000원","289,000",https://kream.co.kr/products/33544
